In [1]:
pwd = %pwd

import os, sys
sys.path.insert(0, os.path.join(pwd, "../"))


#

-------

自作の強化学習アルゴリズムを作成するには以下のクラスを定義します。  
  
Config  
RemoteMemory  
Parameter  
Trainer  
Worker  

ここでは一番簡単なテーブル型のQ学習を自作してみます。

In [2]:
from typing import Any, List,  Dict, Union, cast


# Config

---
強化学習アルゴリズムの種類やハイパーパラメータ等を管理するクラスです。  
RLConfigを継承して作成してもいいですが、強化学習の種類によるインタフェースも提供しています。  
'srl.rl.algorithms'

In [3]:
from dataclasses import dataclass
from srl.base.rl.algorithms.table import TableConfig

@dataclass
class MyConfig(TableConfig):
    
    # ハイパーパラメータ
    epsilon: float = 0.1
    test_epsilon: float = 0
    gamma: float = 0.9
    lr: float = 0.1

    # 親クラスのコンストラクタも呼び出してください
    def __post_init__(self):
        super().__init__()

    # 名前だけユニークなものを指定
    @staticmethod
    def getName() -> str:
        return "MyRL"


# RemoteMemory

---
Workerが取得した経験(batch)をTrainerに渡す役割を持っているクラスです。  
分散学習では multiprocessing のサーバプロセス(Manager)になります。  
ですので、変数へのアクセスができなくなる点だけ制約があります。（全て関数経由でやり取りする必要があります）  
  

In [4]:
from srl.base.rl.base import RLRemoteMemory

class MyRemoteMemory(RLRemoteMemory):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)
        # self.config は上で定義した MyConfig が入っています
        self.config = cast(MyConfig, self.config)

        # 経験を一時的に保存する用
        self.buffer = []

    # (abstractmethod) メモリに保存されている数を返す
    def length(self) -> int:
        return len(self.buffer)

    # (abstractmethod) restore/backupで復元できるように作成
    def restore(self, data: Any) -> None:
        self.buffer = data

    def backup(self):
        return self.buffer

    # --- 以下は独自に定義している関数です

    def add(self, batch: Any) -> None:
        self.buffer.append(batch)

    def sample(self):
        buffer = self.buffer
        self.buffer = []
        return buffer


またよくあるクラスは 'srl.rl.remote_memoty' に定義しています。  
経験を順番通りに取り出す SequenceRemoteMemory を使う場合は以下です。  


In [5]:
from srl.base.rl.remote_memory import SequenceRemoteMemory

class MyRemoteMemory(SequenceRemoteMemory):
    pass


# Parameter

---
学習する/したパラメータを管理するクラスです。  
ある意味強化学習の本体でもあります。  


In [6]:
import json
from srl.base.rl.base import RLParameter

import numpy as np

class MyParameter(RLParameter):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)
        # self.config は上で定義した MyConfig が入っています
        self.config = cast(MyConfig, self.config)

        # Q学習用のテーブル
        self.Q = {}

    # (abstractmethod) restore/backupでQテーブルを復元できるように作成
    def restore(self, data: Any) -> None:
        self.Q = json.loads(data)

    def backup(self):
        return json.dumps(self.Q)

    # Q値を取得する関数
    def get_action_values(self, state: str, invalid_actions):
        if state not in self.Q:
            self.Q[state] = [-np.inf if a in invalid_actions else 0 for a in range(self.config.nb_actions)]
        return self.Q[state]


# Worker

---
実際に環境と連携して経験を収集するクラスです。  
役割は、Parameterを参照してアクションを決める事と、経験をMemoryに送信する事です。  
  
RLWorkerをそのまま継承しても問題ありませんが余分な情報があるので、  
アルゴリズムのタイプ毎にインタフェースを別途定義しています。  
'srl.rl.algorithms' のコードを参照してください。  
  
また、RLWorkerはエピソード内で同じインスタンスが使いまわされます。  
フローをすごく簡単に書くと以下です。  

``` python
env.reset()
worker.on_reset()
while:
    action = worker.policy()
    env.step(action)
    worker.on_step()
```

In [8]:
import random
from srl.base.rl.algorithms.table import TableWorker
from srl.base.env.base import EnvBase

class MyWorker(TableWorker):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)

        # 上で定義した config,parameter,remote_memory が入ります
        self.config = cast(MyConfig, self.config)
        self.parameter = cast(MyParameter, self.parameter)
        self.remote_memory = cast(MyRemoteMemory, self.remote_memory)

    # エピソードの最初に呼ばれる関数
    # state: 環境の初期状態が入っています
    # invalid_actions: 有効でないアクションのリストです
    def call_on_reset(self, state: np.ndarray, invalid_actions: List[int]) -> None:
        # 初期状態を保存
        self.state = str(state.tolist())
        self.invalid_actions = invalid_actions

        if self.training:
            self.epsilon = self.config.epsilon
        else:
            self.epsilon = self.config.test_epsilon

    # 戦略部分です。アクションを返します。
    # state: 環境の状態が入っています
    # invalid_actions: 有効でないアクションのリストです
    # 戻り値はアクションです。
    def call_policy(self, state: np.ndarray, invalid_actions: List[int]) -> int:
        # アクション前の状態を保存
        self.state = str(state.tolist())
        self.invalid_actions = invalid_actions

        if random.random() < self.epsilon:
            # epsilonより低いならランダムに移動
            action = random.choice([a for a in range(self.config.nb_actions) if a not in invalid_actions])
        else:
            q = self.parameter.get_action_values(self.state, invalid_actions)
            q = np.asarray(q)

            # 最大値を選ぶ（複数あればランダム）
            action = random.choice(np.where(q == q.max())[0])

        self.action = int(action)
        return self.action

    # 1step毎に呼ばれる関数です
    # next_state: アクション実行後の状態
    # reweard   : アクション実行後の報酬
    # done      : アクション実行後の終了状態
    # next_invalid_actions: アクション実行後の有効でないアクションのリスト
    def call_on_step(
        self,
        next_state: Any,
        reward: float,
        done: bool,
        next_invalid_actions: List[int],
    ) -> Dict[str, Union[float, int]]:
        if not self.training:
            return {}
        
        batch = {
            "state": self.state,
            "next_state": str(next_state.tolist()),
            "action": self.action,
            "reward": reward,
            "done": done,
            "invalid_actions": self.invalid_actions,
            "next_invalid_actions": next_invalid_actions,
        }
        self.remote_memory.add(batch)
        return {}

    # 強化学習の可視化用
    # 今回ですと、Qテーブルを表示しています。
    def render(self, env: EnvBase) -> None:
        q = self.parameter.get_action_values(self.state, self.invalid_actions)
        maxa = np.argmax(q)
        for a in range(self.config.nb_actions):
            if a == maxa:
                s = "*"
            else:
                s = " "
            s += f"{env.action_to_str(a)}: {q[a]:7.5f}"
            print(s)


# Trainer

---
学習を定義する部分です。  
RemoteMemory から経験を受け取ってParameterを更新します。  


In [9]:
from srl.base.rl.base import RLTrainer

class MyTrainer(RLTrainer):

    # init の引数は親クラスにそのまま渡します。
    def __init__(self, *args):
        super().__init__(*args)

        # 上で定義した config,parameter,memory が入ります
        self.config = cast(MyConfig, self.config)
        self.parameter = cast(MyParameter, self.parameter)
        self.remote_memory = cast(MyRemoteMemory, self.remote_memory)

        self.train_count = 0

    # (abstractmethod) 学習回数を返します
    def get_train_count(self):
        return self.train_count

    # (abstractmethod)
    def train(self):

        # memoryから経験を取得する
        batchs = self.remote_memory.sample()
        td_error = 0
        for batch in batchs:
            # 各batch毎にQテーブルを更新する

            s = batch["state"]
            n_s = batch["next_state"]
            action = batch["action"]
            reward = batch["reward"]
            done = batch["done"]
            invalid_actions = batch["invalid_actions"]
            next_invalid_actions = batch["next_invalid_actions"]

            q = self.parameter.get_action_values(s, invalid_actions)
            n_q = self.parameter.get_action_values(n_s, next_invalid_actions)

            if done:
                target_q = reward
            else:
                target_q = reward + self.config.gamma * max(n_q)

            td_error = target_q - q[action]
            q[action] += self.config.lr * td_error

            td_error += td_error
            self.train_count += 1

        if len(batchs) > 0:
            td_error /= len(batchs)
        
        # 学習結果の情報を返す(任意)
        return {
            "Q": len(self.parameter.Q),
            "td_error": td_error,
        }


# 登録

最後にこれらのクラスを登録します。

In [10]:
from srl.base.rl.registration import register
register(
    MyConfig,
    __name__ + ":MyRemoteMemory",
    __name__ + ":MyParameter",
    __name__ + ":MyTrainer",
    __name__ + ":MyWorker",
)


# テスト

In [11]:
from srl.test import TestRL

tester = TestRL()
tester.play_sequence(MyConfig())
#tester.play_mp(MyConfig()) # notebookだと無限ループする TODO



### env: FrozenLake-v1, max episodes: -1, max steps: 10, timeout:  -1.00s
16:30:36   0.00s       4ep      11tr  -0.00s(remain), 0.000 0.000 0.000 reward, 2.2 step, 0.00s/ep, 0.000s/tr,        0 mem|Q 4.875|td_error 0.000
### 0

SFFF
FHFH
FFFH
HFFG

*0: 0.00000
 1: 0.00000
 2: 0.00000
 3: 0.00000
### 1, done: False
player 0, action 3, reward: 0.0
  (Up)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.00000
 1: 0.00000
 2: 0.00000
 3: 0.00000
### 2, done: False
player 0, action 2, reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.00000
 1: 0.00000
 2: 0.00000
 3: 0.00000
### 3, done: False
player 0, action 2, reward: 0.0
  (Right)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.00000
 1: 0.00000
 2: 0.00000
 3: 0.00000
### 4, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {

# 実行

In [12]:
import srl
from srl.runner import sequence
from srl.runner.callbacks import PrintProgress

config = sequence.Config(
    env_config=srl.envs.Config("FrozenLake-v1"),
    rl_config=MyConfig(),
)

# --- train
config.set_train_config(max_episodes=10000, callbacks=[PrintProgress()])
parameter, remote_memory = sequence.train(config)

# --- test
config.set_play_config(max_episodes=100)
rewards, _, _ = sequence.play(config, parameter)
print("100エピソードの平均結果", np.mean(rewards))


### env: FrozenLake-v1, max episodes: 10000, max steps: -1, timeout:  -1.00s
16:30:41   5.00s    1030ep   18024tr  10.04s(remain), 0.000 0.108 1.000 reward, 17.5 step, 0.00s/ep, 0.237 val_reward, 0.000s/tr,        0 mem|Q 15.915|td_error 0.004
16:30:51  15.00s    2808ep   56600tr  10.74s(remain), 0.000 0.265 1.000 reward, 21.7 step, 0.00s/ep, 0.488 val_reward, 0.000s/tr,        0 mem|Q 16.000|td_error 0.003
16:31:11  35.02s    6402ep  136177tr   5.61s(remain), 0.000 0.272 1.000 reward, 22.2 step, 0.00s/ep, 0.456 val_reward, 0.000s/tr,        0 mem|Q 16.000|td_error 0.003
16:31:32  35.02s   10000ep  216239tr   0.00s(remain), 0.000 0.270 1.000 reward, 22.3 step, 0.00s/ep, 0.535 val_reward, 0.000s/tr,        0 mem|Q 16.000|td_error 0.003
100エピソードの平均結果 0.33


In [13]:
from srl.runner.callbacks import Rendering

render = Rendering(step_stop=False, enable_animation=True)
config.set_play_config(max_episodes=1, callbacks=[render])
rewards, _, _ = sequence.play(config, parameter)


### 0

SFFF
FHFH
FFFH
HFFG

*0: 0.05272
 1: 0.05163
 2: 0.05162
 3: 0.05254
### 1, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.05272
 1: 0.05163
 2: 0.05162
 3: 0.05254
### 2, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.05272
 1: 0.05163
 2: 0.05162
 3: 0.05254
### 3, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.05272
 1: 0.05163
 2: 0.05162
 3: 0.05254
### 4, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.3333333333333333}
work_info 0: {}
train_info: None
*0: 0.05272
 1: 0.05163
 2: 0.05162
 3: 0.05254
### 5, done: False
player 0, action 0, reward: 0.0
  (Left)
SFFF
FHFH
FFFH
HFFG

env_info  : {'prob': 0.33333333

In [14]:
# render.create_anime(fps=2).save("MyRL-FrozenLake.gif")
render.display(fps=2)
